# Fitting to time tag data natively

This notebook is intended to give an example of how to fit an estimated photon arrival rate to time tags (without binning).  

This notebook uses simulated data which.  To generate this data, you will need a function in the repo

    NCAR/deadtime-sim-ptv
    
This notebook utilizes PyTorch to perform the fitting routine.

Our aim is to obtain an estimate of $\lambda(t)$, the photon arrival rate, as a function of time $t$ (relative to laser pulse firing).  We obtain this using a list of $N$ time tags where photons arrived $\mathbf{t} =  \lbrace t_n \rbrace_{n=1}^N$.

For a Poisson point process the probability that the random photon arrivals $\mathbf{T}$ correspond the observations is

$\mathbb{P}\left( \lbrace T_n = t_n \rbrace_{n=1}^N \right) = e^{-\Lambda(t_{max})}\prod_{n=1}^{N} \lambda(t_n)$

where

$\Lambda(t) = \int_0^{t} \lambda(t') dt'$

and $t_{max}$ is the maximum time in the retrieval.

The above probability accounts for one laser shot.  For multiple shots, the PDFs should be multiplied.  In that case, for $L$ laser shots we obtain

$\mathbb{P}\left( \lbrace T_n = t_n \rbrace_{n=1}^N \right) = e^{-L \Lambda(t_{max})}\prod_{n=1}^{N} \lambda(t_n)$

where the list of photons comprises all arrivals from all laser shots.

In order to perform a fit, we aim to minimize the negative log-likelihood of the PDF.  This gives the loss function

$\mathcal{L}[\lambda(t)] = L \Lambda(t_{max}) - \sum_{n=1}^{N} \ln\lambda(t_n)$

From the above definition, it becomes clear that it is useful to have basis functions for $\lambda(t)$ that can be integrated analytically.  For this example, we use a Gaussian target in a constant background

$\lambda(t) = A \exp\left( -\frac{(t-\mu)^2}{2\sigma^2}\right) + b$

where the parameters $A$, $\mu$, $\sigma$ and $b$ must be estimated in the fitting routine.  The integral of the above arrival rate basis function is

$\Lambda(t) = \frac{1}{2} A \sigma \sqrt{2\pi} \left[1 + erf\left(\frac{t-\mu}{\sigma\sqrt{2}}\right)\right] + b t$

where $erf$ is the Gauss Error Function.

In [ ]:
import os,sys
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
dtype = torch.float64
device = torch.device("cpu")

In [ ]:
# import library for simulating Poisson point processes
dirP_str = os.path.join(os.environ['HOME'], 
                    'Documents',
                    'GitHub',
                    'deadtime-sim-ptv',
                    'python',
                    'library')
if dirP_str not in sys.path:
    sys.path.append(dirP_str)
    
import sim_deadtime_utils as sim

# Generate Simulated Data

In [ ]:
# simulation resolution settings
t_sim_min = 0
t_sim_max = 1e-6
dt_sim = 1e-12

tD = 200e-9 # deadtime
Nshot = 500 # number of laser shots
wrap_deadtime = True  # wrap deadtime between shots

laser_pulse_width = 20e-9 # laser pulse width in seconds
target_time = 700e-9
target_amplitude = 5e6 # target peak count rate
background = 1e4  # background count rate

In [ ]:
# generate the simulated scene time axis
t_sim = np.arange(t_sim_min,t_sim_max,dt_sim)           # simulation time
t_sim_bins = np.concatenate((t_sim,t_sim[-1:]+dt_sim))  # simulation time histogram bins

In [ ]:
# generate the photon arrival rate of the profile
# Gaussian target with constant background
photon_rate_arr = target_amplitude*np.exp(-(t_sim - target_time)**2/(2*laser_pulse_width**2))+background

In [ ]:
# generate photon counts

# lists of photon arrivals per
# laser shot
t_det_lst = []  # detected photons (includes deadtime)
t_phot_lst = [] # actual photons (no dead time)

t_det_last = -100.0  # last photon detection event
for n in range(Nshot):
    # simulate a laser shot
    ptime,ctime = sim.photon_count_generator(t_sim_bins, 
                                            photon_rate_arr, 
                                            tau_d_flt=tD,
                                            last_photon_flt=t_det_last)
    if wrap_deadtime:
        if len(ctime) > 0:
            t_det_last = ctime[-1]
        t_det_last -= t_sim_bins[-1]
        
    t_det_lst += [ctime]  # detection time tags (including deadtime)
    t_phot_lst += [ptime] # photon time tags

In [ ]:
phot_arr = np.array(sorted(np.concatenate(t_phot_lst)))
plt.figure()
plt.stem(phot_arr,np.ones(phot_arr.size))
plt.title('Photons')


cnt_arr = np.array(sorted(np.concatenate(t_det_lst)))
plt.figure()
plt.stem(cnt_arr,np.ones(cnt_arr.size))
plt.title('Detected Photons')

In [ ]:
# split data into fit and validation sets
# since we are assuming a fixed target between shots
# we can just split the data in chunks.
split_index = int(len(t_phot_lst)//2)
cnt_phot_fit = len(t_phot_lst[:split_index])
cnt_phot_val = len(t_phot_lst[split_index:])
t_phot_fit = np.concatenate(t_phot_lst[:split_index])
t_phot_val = np.concatenate(t_phot_lst[split_index:])

Double check the integral approximation using erf()

In [ ]:
t_torch = torch.tensor(t_sim)
integral_est = torch.tensor(t_sim*background) \
        + 0.5*target_amplitude*laser_pulse_width*np.sqrt(2*np.pi) \
        + 0.5*target_amplitude*laser_pulse_width*np.sqrt(2*np.pi)\
            *torch.special.erf((t_torch-target_time)/(laser_pulse_width*np.sqrt(2)))

In [ ]:
plt.figure()
plt.plot(t_sim,photon_rate_arr)

plt.figure()
plt.plot(t_sim,np.cumsum(photon_rate_arr)*dt_sim)
plt.plot(t_sim,integral_est.numpy(),'--')

# Perform Fit Using PyTorch
Assume the target is described by a Gaussian with a constant background

In [ ]:
# convert training and validation data to 
# tensor type
t_phot_fit_tnsr = torch.tensor(t_phot_fit)
t_phot_val_tnsr = torch.tensor(t_phot_val)

In [ ]:
# build the fit model as a NN module
class Fit_Pulse(torch.nn.Module):
    def __init__(self,t_max):
        """
        Instantiate and initialize the fit parameters.
        """
        super().__init__()
        self.log_A = torch.nn.Parameter(5*torch.ones(()))   # Gaussian amplitude
        self.log_mu = torch.nn.Parameter(-9*torch.ones(()))  # Gaussian mean
        self.log_sig = torch.nn.Parameter(-9*torch.ones(())) # Gaussian std
        self.log_b = torch.nn.Parameter(3*torch.ones(()))      # background
        self.t_max = t_max # maximum time evaluated in integral term
        
        self.sqrt_2pi = torch.sqrt(torch.tensor(2*np.pi))
        self.sqrt_2  = torch.sqrt(torch.tensor(2.0))
        

    def forward(self, t):
        """
        Forward model the profile for input time t.
        Also return the integral.
        """
        # calculate all the actual fit parameters
        A = torch.exp(self.log_A)
        mu = torch.exp(self.log_mu)
        sig = torch.exp(self.log_sig)
        b = torch.exp(self.log_b)
        
        # calculate the forward model
        model_out = A*torch.exp(-(t-mu)**2/(2*sig**2)) + b
        
        # calculate the integral
        integral_out = b*self.t_max \
                        + 0.5*A*sig*self.sqrt_2pi \
                        + 0.5*A*sig*self.sqrt_2pi\
                            *torch.special.erf((self.t_max-mu)/(sig*self.sqrt_2))
        
        return model_out, integral_out

In [ ]:
def pois_loss(prof,integral):
    """
    Poisson point process loss function
    prof: estimated photon arrival rate evaluated at
        each time tag
    integral: integral of prof at t_max
    """
    return integral-torch.sum(torch.log(prof))

In [ ]:
# optimization parameters
rel_step_lim = 1e-15  # termination criteria based on step size
max_epochs = 5000     # maximum number of iterations/epochs
learning_rate = 1e-1  # ADAM learning rate
term_persist = 20     # relative step size averaging interval in iterations

In [ ]:
# initialize for fit loop
fit_model = Fit_Pulse(t_sim[-1])
optimizer = torch.optim.Adam(fit_model.parameters(),lr=learning_rate)
epoch=0
rel_step = 1e3*rel_step_lim
train_loss_lst = []
valid_loss_lst = []
rel_step_lst = []

In [ ]:
# set the loss function to use a 
# Poisson point process likelihood function
loss_fn = pois_loss

In [ ]:
# perform fit
while rel_step > rel_step_lim and epoch < max_epochs:
    fit_model.train()
    pred,integral = fit_model(t_phot_fit_tnsr)
    loss = loss_fn(pred,integral*cnt_phot_fit)  # add regularization here
    train_loss_lst +=[loss.item()]

    # calculate relative step as an average over the last
    # term_persist iterations
    if epoch == 0:
        rel_step_lst+=[1e3*rel_step_lim]
        rel_step = 1e3*rel_step_lim
    else:
        rel_step_lst+=[(train_loss_lst[-2]-train_loss_lst[-1])/np.abs(train_loss_lst[-2])]
        rel_step = np.abs(np.array(rel_step_lst)[-term_persist:].mean())
    
    # calculate validation loss... for fun
    pred_val,integral_val = fit_model(t_phot_val_tnsr)
    loss_val = loss_fn(pred_val,integral_val*cnt_phot_val)
    valid_loss_lst +=[loss_val.item()]

    # update estimated parameters
    loss.backward()
    optimizer.step()
    
    # zero out the gradient for the next step
    optimizer.zero_grad()

#     print(f"{train_loss_lst[-1]}")
    
    epoch+=1

In [ ]:
# calculate the validation loss.
# we would use this to determine optimimal tuning parameters
# if we had any (e.g. more Gaussian targets)
pred_val,integral_val = fit_model(t_phot_val_tnsr)
loss_val = loss_fn(pred_val,integral_val*cnt_phot_val)
print(f"Validation Loss: {loss_val}")

In [ ]:
# plot the loss over iterations
plt.figure()
plt.plot(train_loss_lst,label='fit loss')
plt.plot(valid_loss_lst,label='validation loss')
plt.xlabel('fit iteration')
plt.ylabel('Poisson NLL')
plt.legend()

Display the resulting estimate

In [ ]:
pred_mod_full,integral_full = fit_model(torch.tensor(t_sim))

In [ ]:
plt.figure()
plt.plot(t_sim*1e9,photon_rate_arr*1e-6,label='Actual')
plt.plot(t_sim*1e9,pred_mod_full.detach().numpy()*1e-6,'--',label='Fit') # /cnt_phot_fit
plt.xlabel('time [ns]')
plt.ylabel('Photon Arrival Rate [MHz]')
plt.legend()

In [ ]:
# calculate the modeled values at each photon arrival
pred_mod_fit_pts,integral_fit_pts = fit_model(t_phot_fit_tnsr)
pred_mod_val_pts,integral_val_pts = fit_model(t_phot_val_tnsr)

In [ ]:
plt.figure()
plt.plot(t_sim*1e9,photon_rate_arr*1e-6,label='Actual',color='k',alpha=0.3)
plt.scatter(t_phot_fit_tnsr.detach().numpy()*1e9,pred_mod_fit_pts.detach().numpy()*1e-6,s=2,label='Fit')
plt.scatter(t_phot_val_tnsr.detach().numpy()*1e9,pred_mod_val_pts.detach().numpy()*1e-6,s=2,label='Validation',c='tab:red')
plt.xlabel('time [ns]')
plt.ylabel('Photon Arrival Rate [MHz]')
plt.legend()